In [ ]:
# Setup
%pip install -e '.[kafka-python]'

import json
import uuid
from datetime import datetime
from diaspora_event_sdk import Client as GlobusClient
from diaspora_event_sdk.sdk.kafka_client import KafkaProducer, KafkaConsumer

# os.environ["DIASPORA_SDK_ENVIRONMENT"] = "local"
c = GlobusClient()

In [ ]:
# Display current user subject
print(f"Current user subject: {c.subject_openid}")

In [ ]:
# 1. Create User
user_result = c.create_user()
print(json.dumps(user_result, indent=2, default=str))

In [ ]:
# Display namespace (automatically computed from subject)
print(f"Using namespace: {c.namespace}")

In [ ]:
# 2. Create Key
key_result = c.create_key()
print(json.dumps(key_result, indent=2, default=str))

In [ ]:
# 3. List namespaces
namespaces_result = c.list_namespaces()
print(json.dumps(namespaces_result, indent=2, default=str))

In [ ]:
# 4. Create Topic
topic_name = f"topic-{str(uuid.uuid4())[:5]}"
create_topic_result = c.create_topic(topic_name)
print(json.dumps(create_topic_result, indent=2, default=str))

In [ ]:
# Kafka topic name format: "{namespace}.{topic}"
kafka_topic = f"{c.namespace}.{topic_name}"
print(f"Kafka topic name: {kafka_topic}")

In [ ]:
# 5. Produce Messages
p = KafkaProducer(kafka_topic)
for i in range(3):
    message = {
        "message_id": i + 1,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "content": f"Message {i + 1} from v3 API",
    }
    future = p.send(kafka_topic, message)
    result = future.get(timeout=30)
    print(f"Produced message {i + 1}: offset={result.offset}")
p.close()

In [ ]:
# 6. Consume Messages
consumer = KafkaConsumer(kafka_topic, auto_offset_reset="earliest")
messages = consumer.poll(timeout_ms=10000)
for tp, msgs in messages.items():
    for message in msgs:
        data = json.loads(message.value.decode("utf-8"))
        print(f"Consumed: {data}")
consumer.close()

In [ ]:
# 7. Recreate Topic
recreate_result = c.recreate_topic(topic_name)
print(json.dumps(recreate_result, indent=2, default=str))

In [ ]:
# 8. Delete Topic
delete_topic_result = c.delete_topic(topic_name)
print(json.dumps(delete_topic_result, indent=2, default=str))

In [ ]:
# 9. Delete Key
delete_key_result = c.delete_key()
print(json.dumps(delete_key_result, indent=2, default=str))

In [ ]:
# 10. Delete User
delete_user_result = c.delete_user()
print(json.dumps(delete_user_result, indent=2, default=str))